In [ ]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations

In [ ]:
! git clone https://github.com/parth1620/Human-segmentation-Dataset-master.git

In [ ]:
import sys
sys.path.append('/contect/Human-Segmentation-Dataset-master')


In [ ]:
import torch
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import helper

In [ ]:
CSV_FILE="/content/Human-segmentation-Dataset-master/train.csv"
DATA_DIR="/content/"
DEVICE="cuda"
EPOCHS=25
LR=0.003
BATCH_SIZE=16
IMG_SIZE=320
ENCODER='timm-efficientnet-b0'
WEIGHTS='imagenet'

In [ ]:
df=pd.read_csv(CSV_FILE)
df.head()

In [ ]:
row=df.iloc[1]
image_path=row.images
mask_path=row.masks

In [ ]:
image=cv2.imread(image_path)
print(image)

In [ ]:
image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
mask=cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)/255.0

In [ ]:
f, (ax1,ax2)=plt.subplots(1 , 2, figsize=(10,5))
ax1.set_title("IMAGE")
ax1.imshow(image)
ax2.set_title("MASK")
ax2.imshow(mask)

In [ ]:
train_df, valid_df= train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
import albumentations as A

In [ ]:
def get_train_augs():
  return A.compose([
      A.Resize(IMG_SIZE, IMG_SIZE),
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.5)
  ])

In [ ]:
def get_valid_augs():
  return A.compose([
      A.Resize(IMG_SIZE, IMG_SIZE)
  ])

## Creating Custom Dataset

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class  SegmentationDataset(Dataset):
     def  __init__(self, df, augmentations):
         self.df=df
         self.augmentations=augmentations

     def  __len__(self):
         return len(self.df)

     def  __getitem__(self, idx):
         row=self.df.iloc[idx]
         image_path=row.images
         mask_path=row.masks
         image=cv2.imread(image_path)
         image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
         mask=cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
         mask=np.expand_dims(mask, axis=-1)
     if  self.augmentations:
         data=self.augmentations(image=image, mask=mask)
         image=data["image"]
         mask=data["mask"]
     image=np.transpose(image, (2,0,1)).astype(np.float32)
     mask=np.transpose(mask, (2,0,1)).astype(np.float32)
     image=torch.Tensor(image)/255.0
     mask=torch.round(torch.Tensor(mask)/255.0)
     return image, mask

In [ ]:
trainset=SegmentationDataset(train_df, get_train_agus())
validset=SegmentationDataset(valid_df, get_valid_agus())

In [ ]:
print(f" Size of Trainset: {len(trainset)}")
print(f" Size of Validset: {len(validset)}")

In [ ]:
idx=3
image , mask= trainset[idx]
helper.show_image(image, mask)

#Load Dataset into Batches

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainloader=DataLoader(trainset, batch_size=BATCH_SIZE)
validloader=DataLoader(validset, batch_size=BATCH_SIZE)

In [ ]:
print(f"Total numbers of batches in trainloader: {len(trainloader)}")
print(f"Total numbers of batches in validloader: {len(validloader)}")

In [ ]:
for image, mask in trainloader:
  break

In [ ]:
print(f" One batch image shape: {image.shape}")
print(f" One batch mask shape: {mask.shape}")

# Create Segmentation model

In [ ]:
from torch import nn

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
from segmentation_models_pytorch.losses import DiceLoss

In [ ]:
from torch.nn.modules.conv import LazyConvTranspose2d
from cv2.dnn import SegmentationModel
class SegmentationModel(nn.Module):
  def __init__(self):
    super(SegmentationModel, self).__init__()
    self.arc=smp.Unet(
        encoder_name=ENCODER,
        encoder_weights=WEIGHTS,
        in_channels=3,
        classes=1,
        activation=None
    )
  def forward(self, images, masks=None):
    logits=self.arc(images)
    if mask != None:
      loss1=DiceLoss(mode="binary")(logits, masks)
      loss2=nn.BCEWithLogitsLoss()(logits, masks)
      return logits, loss1+loss2
    return logits

In [ ]:
model=SegmentationModel()
model.to(DEVICE);

# create train and validation functions

In [ ]:
def train_fn(data_loader, model, optimizer):
  model.train()
  total_loss=0.0
  for images, masks in tqdm(data_loader):
    images=images.to(DEVICE)
    masks=masks.to(DEVICE)

    optimizer.zero_grad()
    logits , loss=model(images, masks)
    loss.backward()
    optimizer.step()

    total_loss+= loss.item()
  return total_loss/len(data_loader)

In [ ]:
def eval_fn(data_loader, model):
  model.eval()
  total_loss=0.0
  with torch.no_grad():
    for images, masks in tqdm(data_loader):
      images=images.to(DEVICE)
      masks=masks.to(DEVICE)

      logits , loss=model(images, masks)

      total_loss += loss.item()
  return total_loss/len(data_loader)

# Train The Model

In [ ]:
optimizer=torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
best_valid_loss=np.Inf
for i in range(EPOCHS):
  train_loss=train_fn(trainloaader, model, optimizer)
  valid_loss=eval_fn(validloader, model)
  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), "best_model.pt")
    print("SAVE MODEL")
    best_valid_loss=valid_loss
  print(f" Epoch :{i+1} Train_loss :{train_loss} Valid_loss:{valid_loss}")

# inference
we are evaluating our model with validset

In [ ]:
idx=20
model.load_state_dict(torch.load("/content/best_model.pt"))
image, mask= validset[idx]
logits_mask=model(image.to(DEVICE), unsqueeze(0))
pred_mask=torch.sigmoid(logits_mask)
pred_mask=(pred_mask> 0.5)* 1.0



In [ ]:
helper.show_image(image, mask, pred_mask.detach().cpu().squeeze(0))